In [1]:
! pip install konlpy

     |████████████████████████████████| 19.4MB 1.4MB/s 
     |████████████████████████████████| 460kB 29.1MB/s 
     |████████████████████████████████| 92kB 8.5MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [4]:
from konlpy.tag import Komoran
import re
komoran = Komoran(userdic='/content/drive/MyDrive/models/dict.txt')

def tokenizer(text):
    # josa = ['JKS', 'JKC', 'JKG', 'JKO', 'JKB', 'JKV', 'JKQ', 'JX', 'JC', 'EP', 'EF', 'EC', 'ETN', 'ETM', 'XPN', 'XSN', 'XSV', 'XSA', 'XR']
    pos_tag = ['NNG', 'NNP', 'NNB', 'NP', 'NR', 'VA', 'MM', 'MAG']
    tokens_pos = komoran.pos(text)
    tokens_word = [] 
    for tag in tokens_pos:
        if tag[1] in pos_tag:
            tokens_word.append(tag[0])
    return re.sub('\.','',' '.join(tokens_word))

In [5]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/spell_check_label.csv')

In [6]:
dataset = []
for sent in df['fixed']:
  dataset.append(tokenizer(sent).split())

In [7]:
from gensim import models
w2v = models.Word2Vec(dataset,sg=1, window=8, min_count=1)
w2v.init_sims(replace=True)

In [8]:
def getKeywords(file):
  df = pd.read_csv(file)
  keywords = df.iloc[:30, 0]
  keywords = keywords.values.tolist()
  return keywords

In [9]:
def group(keywords):
  groups = []
  for word in keywords:
   sim = []
   for i in w2v.wv.most_similar(word, topn=20):
     sim.append(i[0])
   group_words = [word]
   keywords.remove(word)
   for i in keywords:
      if i in sim:
        group_words.append(i)
        keywords.remove(i)
   groups.append(group_words)
  return groups

In [10]:
def identity_tokenizer(text):
    return text

In [11]:
def getHotelname(path):
  name = path[5:]
  name = name[:-4]
  return name

In [12]:
def getRep(groups):

  from sklearn.feature_extraction.text import TfidfVectorizer

  tfidfv = TfidfVectorizer(tokenizer=identity_tokenizer,lowercase=False)    
  X = tfidfv.fit_transform(dataset)
  tfidf = X.toarray()

  rep = []

  for i in range(len(groups)):

    if len(groups[i])>1:
      ar=[0]*len(groups)
      for j in range(len(groups[i])):
        c2 = 0
        for c in dataset:
          for c1 in groups[i]:
            if c1 in c:
              c2+=1
              break
        z=tfidfv.vocabulary_[groups[i][j]]
        t=0
        for j1 in range(len(tfidf)):
          t+=tfidf[j1][z]
        t=t/c2
        t1=0
        for j1 in range(len(groups[i])):
          if j1 != j:
            t+=w2v.wv.similarity(w1=groups[i][j], w2=groups[i][j1])
        t1=t1/(len(groups[i])-1)
        ar[j]=t+t1
      rep.append(groups[i][ar.index(max(ar))])
    else:
      rep.append(groups[i][0])

  return rep

In [13]:
import os
path = "/content/drive/MyDrive/key"

hotel_keyword_list = []
hotels = os.listdir(path)
for hotel in hotels:
      keywords = getKeywords(os.path.join(path, hotel))
      groups = group(keywords)
      rep = getRep(groups)
      for i in range(len(rep)):

        hotel_keyword_list.append([getHotelname(hotel), rep[i], groups[i]])
hotel_keyword = pd.DataFrame(hotel_keyword_list,columns=['Hotel','Rep','Keywords'])
hotel_keyword.to_csv('hotel_keyword.csv')